In [9]:
!export PYTHONPATH=$PYTHONPATH:$PWD/..:$PWD

In [1]:
import torch
from hydra.utils import instantiate
from hydra import initialize, compose
import hydra
import matplotlib.pyplot as plt

from PhaseClassifier import logging
logger = logging.getLogger(__name__)

from PhaseClassifier.scripts.run import setup_model

[01:16:48.731] INFO   PhaseClassifier                                   Loading configuration.


In [2]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="PhaseClassifier/config")
config=compose(config_name="config.yaml")

In [3]:
self = setup_model(config)

[01:16:49.595] INFO   data.dataManager                                  Loading dataset: Ising
[01:16:49.597] INFO   data.dataManager                                  Keys: ['Label', 'Snapshot', 'Temperature']


[01:16:49.722] INFO   data.dataManager                                  dict_keys(['Label', 'Snapshot', 'Temperature'])
[01:16:49.763] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f03b499fa10>: 32000 events, 64 batches
[01:16:49.763] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f03b44c9090>: 4000 events, 4 batches
[01:16:49.764] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7f03b44c8e10>: 4000 events, 4 batches
[01:16:49.765] INFO   model.modelCreator                                ::Creating Model
[01:16:49.768] INFO   PhaseClassifier.scripts.run                       Requesting GPUs. GPU list :[2]
[01:16:49.768] INFO   PhaseClassifier.scripts.run                       Main GPU : cuda:2
[01:16:49.935] INFO   PhaseClassifier.scripts.run                       CUDA available


cuda:2


In [10]:
x = next(iter(self.data_mgr.train_loader))

In [11]:
# x[0].shape
pred = self.model(x[0].to(self.device))

In [13]:
import torch.nn.functional as F

F.cross_entropy(pred, x[1][:,0].to(self.device)).item()

0.6930362582206726

In [17]:
self.eval_model(self.data_mgr.val_loader, 0)

[01:18:17.463] INFO   engine.engine                                     Epoch: 0 	 Batch Loss: 0.0054


In [16]:
self.fit_model(0)

[01:18:13.934] INFO   engine.engine                                     Epoch: 0 [0/64 (0%)]	 Batch Loss: 0.0045
[01:18:13.982] INFO   engine.engine                                     Epoch: 0 [6/64 (9%)]	 Batch Loss: 0.0055
[01:18:14.024] INFO   engine.engine                                     Epoch: 0 [12/64 (19%)]	 Batch Loss: 0.0053
[01:18:14.069] INFO   engine.engine                                     Epoch: 0 [18/64 (28%)]	 Batch Loss: 0.0067
[01:18:14.107] INFO   engine.engine                                     Epoch: 0 [24/64 (38%)]	 Batch Loss: 0.0047
[01:18:14.142] INFO   engine.engine                                     Epoch: 0 [30/64 (47%)]	 Batch Loss: 0.0032
[01:18:14.179] INFO   engine.engine                                     Epoch: 0 [36/64 (56%)]	 Batch Loss: 0.0056
[01:18:14.214] INFO   engine.engine                                     Epoch: 0 [42/64 (66%)]	 Batch Loss: 0.0065
[01:18:14.259] INFO   engine.engine                                     Epoch: 0 [48